# Benchmarking cities in MENA

In support of an upcoming Urban flagship report, the MENA team is looking for a series of zonal statistics:

- Nighttime Lights, Population, and built-area:  
  - Entire FUA  
  - Its associated urban center / “core”  
  - Associated “periphery”  

The unit of analysis is the Functional Urban Areas (FUAs) from the [UCDB Database](https://human-settlement.emergency.copernicus.eu/ghs_stat_ucdb2015mt_r2019a.php). For each FUA, we need to grab the associated urban periphary (lower threshold urban areas)

In [ ]:
import sys
import os

import geopandas as gpd


sys.path.append("C:/WBG/Work/Code/GOSTrocks/src")

In [ ]:
data_folder = "C:/WBG/Work/data"
ucdb_file = os.path.join(
    data_folder,
    "URBAN",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg",
)
fua_file = os.path.join(
    data_folder, "URBAN", "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
)

out_folder = "C:/WBG/Work/MENA_Urban/"
urban_res_folder = os.path.join(out_folder, "urban_data")

for out_folder in [urban_res_folder]:
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

urban_periphary_file = os.path.join(urban_res_folder, "urban_periphary.gpkg")
urban_core_file = os.path.join(urban_res_folder, "urban_core.gpkg")

In [ ]:
inD = gpd.read_file(ucdb_file)
inF = gpd.read_file(fua_file)
inD = inD.to_crs(inF.crs)

In [ ]:
inF.head()

In [ ]:
fua_peripheries = inF.copy()
for idx, row in inF.iterrows():
    # grab the related UCDBs
    ucdb_ids = row["UC_IDs"].split(";")
    ucdb_ids = [int(x) for x in ucdb_ids]
    sel_cores = inD.loc[inD["ID_HDC_G0"].isin(ucdb_ids)]
    periphery_geom = row["geometry"].difference(sel_cores.unary_union)
    fua_peripheries.loc[idx, "geometry"] = periphery_geom

fua_peripheries.to_file(os.path.join(out_folder, "FUA_peripheries.gpkg"), driver="GPKG")